# PPT Generator

In [8]:
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()

True

In [10]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [11]:
import google.generativeai as genai

In [12]:
genai.configure(api_key=google_api_key)

In [13]:
for m in genai.list_models():
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/aqa


In [14]:
model = genai.GenerativeModel('gemini-pro')

In [15]:
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [16]:
response = model.generate_content("Give me a Quote from Virat Kohli")

In [17]:
response.text

'"Pressure is a privilege."'

#### List All The DOCX Files


First We Get All the DocX Files from the folder

In [18]:
folder = "./Data/"

In [19]:
import glob

In [20]:
def list_docx_files(folder_path):
    doc_files = glob.glob(os.path.join(folder_path,"*.docx"))
    return doc_files

In [21]:
file = list_docx_files(folder)

In [22]:
import docx

In [ ]:
# Code Referneve to read and store the data in a dataframe.
# def read_docx_files(directory):
#     """
#     Read .docx files from a directory and extract content along with filenames.
#     """
#     data = []
#     for filename in os.listdir(directory):
#         if filename.endswith('.docx'):
#             file_path = os.path.join(directory, filename)
#             document = Document(file_path)
#             content = ""
#             for para in document.paragraphs:
#                 content += para.text + "\n"
#             data.append({'File Name': filename, 'Content': content})
#     return data